In [ ]:
#!pip install -r requirements.txt

In [1]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
import nltk
from nltk.corpus import stopwords
import re

In [2]:
# Definir URL del archivo
url = "https://github.com/tosorio/fakes-technical-interview-dataset/raw/main/Work%20Sample%202025%20-%20Analista%20Sr.%20de%20Operaciones%20%26%20Analytics%20-%20Moderaciones%20IT.xlsx"
# Leer todas las sheets
doc = pd.read_excel(url, sheet_name=None)
# Mostrar sheets disponibles
print(doc.keys())

dict_keys(['Introducción', 'Descripción', 'Base'])


In [3]:
# Print intro
print(doc['Introducción'])

Empty DataFrame
Columns: [Introducción

Como parte del proceso de selección para el puesto de Analista Sr. de Operaciones & Analytics - Moderaciones IT, te proporcionamos un conjunto de datos relacionado con la moderación de productos en MercadoLibre. Tu tarea es explorarlo en profundidad y extraer insights clave que puedan mejorar la detección de productos falsificados.

Algunos aspectos a considerar en tu análisis:
- Descripción y estructura: Familiarízate con los atributos del dataset y su posible relevancia en la detección de falsificaciones.
- Distribución y patrones: Identifica tendencias, anomalías o concentraciones inusuales en las moderaciones.
- Relaciones y correlaciones: Explora cómo se conectan los diferentes factores y qué características podrían ser indicativas de falsificaciones.

Más allá de estos puntos, te invitamos a pensar fuera de la caja. El dataset ofrece oportunidades para proponer mucho mas que esto ¿Qué hallazgos inesperados surgen? ¿Qué estrategias adicional

In [4]:
# Mostrar metadata
meta = doc['Descripción']
meta.columns = meta.iloc[0]
meta = meta.iloc[1:]
meta

,Campo,Descripcion
1,element_id,Id de producto
2,site_id,Pais
3,seller_id,Id del vendedor
4,Dominio_normalizado,Categoria del producto
5,Titulo,Titulo del producto
6,Marca,Marca publicada del producto
7,Precio,Precio del producto
8,Rule,Regla con la que se modero a la publicacion
9,Score,Score del modelo de Machine Learning. Utilizad...
10,Moderado,Flag que indica que el producto fue moderado p...


In [5]:
# Mostrar dataset
df = doc['Base'].copy()
# Drop campo no usado
df.drop(columns=['Total'], inplace=True)
df

,element_id,site_id,seller_id,Dominio_normalizado,Titulo,Marca,Precio,Rule,Score,Moderado,Fake,Rollback,FK_TEST1,FK_TEST2
0,ARG1000130348,ARGENTINA,924394736,COMPUTER_PROCESSORS,Microprocesador Intel Core I3 12100 12mb Bx807...,Intel,222599.0,NaN,0.11,0,0,0,0,0
1,ARG1002056504,ARGENTINA,60385780,COMPUTER_MONITORS,Monitor Gamer Samsung Con Pantalla De 25 60hz...,Samsung,767000.0,NaN,0.29,0,0,0,0,0
2,ARG1002630435,ARGENTINA,1085316688,SUNGLASSES,Anteojos De Sol Polarizados Ray-ban Erika Clas...,Ray-Ban,246510.0,NaN,0.51,0,0,0,0,0
3,ARG1003499360,ARGENTINA,189266308,HARD_DRIVES_AND_SSDS,Ssd Externo Kingston Xs1000 1tb Negro Usb 3.2 ...,Kingston,179999.0,NaN,0.32,0,0,0,0,0
4,ARG1004623658,ARGENTINA,127503700,RAM_MEMORY_MODULES,Memoria Ram Valueram Color Verde 8gb 1 Kingsto...,Kingston,22399.0,NaN,0.80,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40367,PER9979201495,PERU,1906413812,SPEAKERS,Bocina Jbl Charge 5 Jblcharge5 Portátil Con Bl...,JBL,799.0,NaN,0.13,0,0,0,0,0
40368,PER9979619481,PERU,699068980,HEADPHONES,Audifonos Jbl Tune 520 Bt Bluetooth On Ear Col...,JBL,299.0,NaN,0.80,0,0,0,0,0
40369,PER9980515950,PERU,1510316564,SMARTWATCHES,Apple Watch Series 10 Gps Caja De Aluminio Ne...,Apple,2499.0,NaN,0.83,0,0,0,0,0
40370,PER999421010,PERU,2495083952,WRISTWATCHES,Reloj Casio W218wd Acero Inoxidable Alarma Sum...,Casio,389.0,NaN,0.10,0,0,0,0,0


# EDA

### Validación de faltantes y agregación individual de productos (```element_id```), paises (```site_id```), vendedores (```seller_id```), categoria de producto (```Dominio_normalizado```) y marca (```Marca```).

In [6]:
# Información general del dataset
df.info()
print()
print(f'- Hay un total de {len(df["element_id"].unique())} productos publicados')
print(f'- En {len(df["Dominio_normalizado"].unique())} categorias')
print(f'- De {len(df["Marca"].unique())} marcas diferentes')
print(f'- A traves de {len(df["site_id"].unique())} paises')
print(f'- Por {len(df["seller_id"].unique())} vendedores')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40372 entries, 0 to 40371
Data columns (total 14 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   element_id           40372 non-null  object 
 1   site_id              40372 non-null  object 
 2   seller_id            40372 non-null  int64  
 3   Dominio_normalizado  40372 non-null  object 
 4   Titulo               40372 non-null  object 
 5   Marca                40372 non-null  object 
 6   Precio               40372 non-null  float64
 7   Rule                 4278 non-null   object 
 8   Score                40372 non-null  float64
 9   Moderado             40372 non-null  int64  
 10  Fake                 40372 non-null  int64  
 11  Rollback             40372 non-null  int64  
 12  FK_TEST1             40372 non-null  int64  
 13  FK_TEST2             40372 non-null  int64  
dtypes: float64(2), int64(6), object(6)
memory usage: 4.3+ MB

- Hay un total de 40372 prod

### Analisis univariado de variables categoricas

In [7]:
# Pie chart redimensionado para presentación
site_df = df.groupby(['site_id']).agg(
    total=('element_id', 'count')
).reset_index()

fig_pie = px.pie(
    site_df,
    names='site_id',
    values='total',
    title="Distribución por país",
    color_discrete_sequence=["#FED609", "#1D2E64", "#76ABBD"]
)
fig_pie.update_layout(
    width=600, height=600,
    font=dict(family="Trebuchet MS", size=20),  # Fuente
    title=dict(text="Distribución por país", x=0.5, font=dict(size=30)),  # Centrar y agrandar título
    legend=dict(orientation="h", yanchor="bottom", y=-0.2, xanchor="center", x=0.5)  # Leyenda
)
fig_pie.show()

In [8]:
# Variables a graficar
fields = ['Dominio_normalizado', 'Marca']

# Crear un DataFrame combinado para el treemap
dominio_marca_df = df.groupby(['Dominio_normalizado', 'Marca']).agg(
    total=('element_id', 'count')
).reset_index()

# Treemap con escala de azules
fig_treemap = px.treemap(
    dominio_marca_df,
    path=['Dominio_normalizado', 'Marca'],
    values='total',
    title="Distribución por categoría y marca",
    color='total',
    color_continuous_scale='Blues'
)
fig_treemap.update_layout(
    title=dict(text="Distribución por categoría y marca", x=0.5, font=dict(size=30)),  # Centrar y agrandar título
    font=dict(family="Trebuchet MS", size=22),  # Fuente
    margin=dict(t=50, l=25, r=25, b=25)  # márgenes
)
fig_treemap.show()

> - Brasil, Argentina y Mexico tienen una participación agregada del 85.3% en el total de productos publicados.
- Dispositivos moviles como Celulares, Relojes y Audifonos encabezan la lista de productos publicados.
- Las marcas de productos de tecnología son las que destacan en la distribución por marca.


### Pasamos a validar variables numericas, iniciamos con el campo de ```Precio```

In [9]:
# Excluir outliers en precio para facilitar la visualización
low, high = df["Precio"].quantile([0.01, 0.99])
df_filtered = df[(df["Precio"] >= low) & (df["Precio"] <= high)]

# Boxplot de precio por país
fig = px.box(df_filtered, x="site_id", y="Precio", color="site_id", title="Distribución de Precio por site_id")
fig.show()

- Los precios estan expresados en la moneda local de cada pais, para continuar el análisis agregado (para fines exploratorios) corresponde llevar los precios a una moneda común de tal forma que sean comparables.

- Por facilidad se define llevar todos los precios a USD a la tasa de cambio fijada por el buscador de Google el dia en el que se efectua e analisis, las tasas de cambio empleadas son:

>1 USD = 1,070.88 ARS <br>
1 USD = 5.76 BRL (No proveido por Google) <br>
1 USD = 20.39 MXN <br>
1 USD = 933.00 CLP <br>
1 USD = 4153.10 COP <br>
1 USD = 3.64 PEN



In [10]:
# Diccionario con tasas de cambio
exchange_rates = {
    "ARGENTINA": 1070.88,
    "BRASIL": 5.76,
    "MEXICO": 20.39,
    "CHILE": 933.00,
    "COLOMBIA": 4153.10,
    "PERU": 3.64
}

# Convertir precios a USD
df["Precio_USD"] = df.apply(lambda row: row["Precio"] / exchange_rates.get(row["site_id"], 1), axis=1)

- Los precios estandarizados se emplean mas adelante en el análisis descriptivo del dataset.

### Analisis de Moderaciones Fakes y Rollbacks

In [11]:
# Llenar faltantes del campo rule
df['Rule'] = df['Rule'].fillna('NO')
# Productos moderados
md = df['Moderado'].value_counts()
mod = (md/len(df)).reset_index()
print(f'º {mod.iloc[1,1]:.2%} ({md.reset_index().iloc[1,1]}) de los productos seleccionados han pasado por filtro de moderación')
# Moderados Fake
mfake = (df[df['Moderado']==1]['Fake'].value_counts()/df['Moderado'].value_counts()[1]).reset_index()
print(f'º De los productos moderados el {mfake.iloc[0,1]:.2%} fue clasificado como Fake en última instancia')
# Incidencia de Rollback
mrolled = (df[df['Moderado']==1]['Rollback'].value_counts()/df['Moderado'].value_counts()[1]).reset_index()
print(f'º De los productos moderados el {mrolled.iloc[1,1]:.2%} fue revertido y reclasificado despues de haber sido clasificado como fake')
# Productos moderados que nunca fueron clasificados como fake
nfke = len(df[(df['Moderado']==1)&(df['Fake']==0)&(df['Rollback']==0)])/len(df[df['Moderado']==1])
print(f'º De los productos moderados el {nfke:.2%} nunca fueron clasificados como fake')

º 10.60% (4278) de los productos seleccionados han pasado por filtro de moderación
º De los productos moderados el 92.38% fue clasificado como Fake en última instancia
º De los productos moderados el 7.15% fue revertido y reclasificado despues de haber sido clasificado como fake
º De los productos moderados el 0.47% nunca fueron clasificados como fake


In [12]:
# Agrupacion de variables
df.groupby(['Moderado', 'Rule', 'Fake', 'Rollback']).agg(
    total=('element_id', 'count'),
    Min_Score=('Score', 'min'),
    Median_Precio_USD=('Precio_USD', 'median')
).reset_index()

,Moderado,Rule,Fake,Rollback,total,Min_Score,Median_Precio_USD
0,0,NO,0,0,35557,0.00,138.715278
1,0,NO,1,0,537,0.86,92.447333
2,1,FK_ATTRIBUTE,0,0,9,0.80,66.215278
3,1,FK_ATTRIBUTE,0,1,71,0.80,69.601275
4,1,FK_ATTRIBUTE,1,0,1187,0.79,133.946181
5,1,FK_MODEL,1,0,432,1.00,87.861011
6,1,FK_PRICE,0,0,11,0.80,471.554684
7,1,FK_PRICE,0,1,235,0.77,93.381144
8,1,FK_PRICE,1,0,2333,0.77,64.062500


- No todos los productos pasan por el filtro de moderación
- Los que son moderados siguen 3 reglas definidas:
  - ```FK_ATTRIBUTE```: Moderación por atributo.
  - ```FK_MODEL```: Moderación por modelo.
  - ```FK_PRICE```: Moderación por precio.
- Solo se presenta ```Rollback``` en moderaciones de ```FK_ATTRIBUTE``` y ```FK_PRICE```.

### Para entender la naturaleza de las moderaciones evaluamos las distibuciones de los datos de score segun las reglas aplicadas ```Rule``` para cada pais

In [13]:
# Calcular cuartiles, máximos y mínimos
summary = df.groupby("Rule")["Score"].describe()[["min", "25%", "75%", "max"]].reset_index()

# Crear el boxplot en orientación horizontal
fig = px.box(
    df,
    y="Rule",
    x="Score",
    color="Rule",
    title="Distribución de scores por regla comparando productos excluidos vs no excluidos",
    orientation='h'
)

# Agregar marcas de cuartiles, máximos y mínimos
for _, row in summary.iterrows():
    rule = row["Rule"]
    min_val, q1, q3, max_val = row[["min", "25%", "75%", "max"]]

    values = [min_val, q1, q3, max_val]
    labels = ["Min", "Q1", "Q3", "Max"]

    for i in range(len(values)):
        # Aplicar marcas
        if i == 0 or abs(values[i] - values[i-1]) > 0.15 * (max_val - min_val):
            fig.add_trace(go.Scatter(
                x=[values[i]],
                y=[rule],
                mode="markers+text",
                marker=dict(size=10, color="black"),
                text=[f"{labels[i]}: {values[i]:.2f}"],  # Etiqueta con nombre y valor
                textposition="top center",
                showlegend=False
            ))
fig.update_layout(
    font=dict(family="Trebuchet MS", size=18),
    title=dict(font=dict(size=28), x=0.5),
    xaxis=dict(title="Score", title_font=dict(size=24)),
    yaxis=dict(title="Regla", title_font=dict(size=24)),
    margin=dict(l=100, r=50, t=80, b=80),
    legend=dict(orientation="h", yanchor="bottom", y=-0.3, xanchor="center", x=0.5)
)
# Mostrar gráfico
fig.show()

- Todos los productos con ```Score``` en el modelo de ML igual a 1 son moderados por ```FK_MODEL``` y son clasificados como ```Fake```.
- Solo productos con score superior a un threshold >~75% son filtrados por ```FK_ATTRIBUTE``` o ```FK_PRICE```.
- Dado que no todos los productos con un score >\~75% son filtrados, se entiende que todos los productos con score >~0.75 y <1 son pasados por reglas de validación adicionales.

### , procedemos a analizar los precios de estos productos para entender la lógica de la regla ```FK_PRICE```


In [14]:
# Filtro de precios por regla
df_ = df[(df['Score'] >= 0.77)&((df['Rule'] == 'FK_PRICE')|(df['Rule'] == 'NO'))]
# Excluir outliers en precio para facilitar la visualización
low, high = df_["Precio_USD"].quantile([0.01, 0.99])
df_filtered = df_[(df_["Precio_USD"] >= low) & (df_["Precio_USD"] <= high)]
# Boxplot de precio por país
fig = px.box(df_filtered, x="site_id", y="Precio_USD", color="Rule", title="Distribución de precio por pais comparando productos excluidos por precio vs no excluidos")
fig.show()

-Parece ser que los precios de los productos descartados por precio tienden a ser menores que en los no descartados.

### Analicemos a mayor detalle si por marca en un país definido este comportamiento se mantiene.

In [15]:
# Filtro de productos por regla listados en Argentina
df_ = df[(df['Score'] >= 0.77)&(df['site_id']=='ARGENTINA')&((df['Rule']=='FK_PRICE')|(df['Rule'] == 'NO'))]
# Excluir outliers en precio para facilitar la visualización
low, high = df_["Precio_USD"].quantile([0.01, 0.99])
df_filtered = df_[(df_["Precio_USD"] >= low) & (df_["Precio_USD"] <= high)]
# Boxplot de precio por Marca
fig = px.box(df_filtered, x="Marca", y="Precio_USD", color="Rule", title="Distribución de precio por marca comparando productos excluidos por precio vs no excluidos en Argentina")
fig.show()

> La distribucion de cada uno de los productos por regla en el país seleccionado contrasta el hallazgo anterior.

### Procedemos a aplicar pruebas estadísticas para validar las diferencias en las distribuciones y medias de los datos de cara a confirmar el hallazgo.

## Hipótesis Estadísticas

### **Prueba t de Student (t-test)**
- **Hipótesis Nula**: No hay diferencia significativa entre los precios de los productos categorizados por ```Marca``` y ```Site_id``` y con valores de ```Rule``` en la categoría ```NO``` (Productos no excluidos) y ```FK_PRICE``` (Productos excluidos por precio). Es decir, ambas muestras provienen de poblaciones con la misma media de precio.  
- **Hipótesis Alternativa**: El precio en la categoría ```FK_PRICE``` es significativamente menor que el precio en la categoría ```NO```.

### **Prueba de Kolmogorov-Smirnov (KS Test)**
- **Hipótesis Nula**: Las distribuciones de precios en las categorías  de los productos categorizados por ```Marca``` y ```Site_id``` y con valores de ```Rule``` en la categoría ```NO``` (Productos no excluidos) y ```FK_PRICE``` (Productos excluidos por precio) son estadísticamente iguales.  
- **Hipótesis Alternativa**: Las distribuciones de precios en las categorías ```NO``` y ```FK_PRICE``` son significativamente diferentes.  

Si los valores p (\(p < 0.05\)) son pequeños en cualquiera de las pruebas, se rechaza la hipótesis nula en favor de la alternativa, lo que indica una diferencia significativa.

In [16]:
# Contar combinaciones únicas de 'site_id' y 'Marca' antes de filtrar
total_comparables = df[['site_id', 'Marca']].drop_duplicates().shape[0]

# Filtrar las combinaciones que tienen al menos una de las reglas 'NO' o 'FK_PRICE'
df_filtered = df[df['Rule'].isin(['NO', 'FK_PRICE'])].copy()

# Contar combinaciones después del primer filtro
comparables_after_rule_filter = df_filtered[['site_id', 'Marca']].drop_duplicates().shape[0]
excluded_by_rule_combinations = total_comparables - comparables_after_rule_filter

# Identificar combinaciones que tienen ambas reglas 'NO' y 'FK_PRICE'
valid_combinations = df_filtered.groupby(['site_id', 'Marca'])['Rule'].nunique()
valid_combinations = valid_combinations[valid_combinations == 2].index

# Filtrar solo combinaciones que tienen ambas reglas
df_valid = df_filtered.set_index(['site_id', 'Marca']).loc[valid_combinations].reset_index()
final_comparables = df_valid[['site_id', 'Marca']].drop_duplicates().shape[0]
excluded_by_na_combinations = comparables_after_rule_filter - final_comparables

# Agrupar por 'site_id', 'Marca' y 'Rule' para calcular estadísticas de precio
df_grouped = df_valid.groupby(['site_id', 'Marca', 'Rule'])['Precio_USD'].agg(['mean', 'median']).unstack()

# Calcular pruebas estadísticas
results = []
for (site, marca), row in df_grouped.iterrows():
    no_prices = df_valid[(df_valid['site_id'] == site) & (df_valid['Marca'] == marca) & (df_valid['Rule'] == 'NO')]['Precio_USD']
    fk_prices = df_valid[(df_valid['site_id'] == site) & (df_valid['Marca'] == marca) & (df_valid['Rule'] == 'FK_PRICE')]['Precio_USD']

    if len(no_prices) < 3 or len(fk_prices) < 3:
        p_valt_uni = np.nan
        ks_stat, p_value_ks = np.nan, np.nan
        conclusion_ttest = "N/A"
    else:
        t_stat, p_valt = stats.ttest_ind(fk_prices, no_prices, equal_var=False, nan_policy='omit')
        p_valt_uni = p_valt / 2 if t_stat < 0 else np.nan
        conclusion_ttest = "FK_PRICE < NO (p < 0.05)" if p_valt_uni < 0.05 else "No diferencia significativa"
        ks_stat, p_value_ks = stats.ks_2samp(fk_prices, no_prices)

    results.append({
        'pais': site,
        'marca': marca,
        'precio_medio_NO': row[('mean', 'NO')],
        'precio_medio_FK_PRICE': row[('mean', 'FK_PRICE')],
        'p_val_ttest_uni': round(p_valt_uni, 4) if not np.isnan(p_valt_uni) else np.nan,
        'Conclusion_ttest': conclusion_ttest,
        'KS_Statistic': round(ks_stat, 4) if not np.isnan(ks_stat) else np.nan,
        'p_value_ks': round(p_value_ks, 4) if not np.isnan(p_value_ks) else np.nan,
        'Significant_KS': 'Pass' if p_value_ks < 0.05 else 'Fail' if not np.isnan(p_value_ks) else 'N/A'
    })

df_results = pd.DataFrame(results)

# Resumen de casos excluidos y resultados finales
summary_df = pd.DataFrame({
    "Metric": [
        "Total Comparables Iniciales",
        "Casos excluidos por 'Rule' ≠ (NO, FK_PRICE)",
        "Casos excluidos por ausencia de ambas reglas",
        "Casos comparados finales",
        "t-test - FK_PRICE < NO",
        "t-test - No diferencia significativa",
        "KS-test - FK_PRICE ≠ NO",
        "KS-test - No diferencia significativa",
        "Mediana p-value (t-test)"
    ],
    "Casos (Participación)": [
        total_comparables,
        excluded_by_rule_combinations,
        excluded_by_na_combinations,
        final_comparables,
        f"{(df_results['Conclusion_ttest'] == 'FK_PRICE < NO (p < 0.05)').sum()} ({(df_results['Conclusion_ttest'] == 'FK_PRICE < NO (p < 0.05)').mean() * 100:.1f}%)",
        f"{(df_results['Conclusion_ttest'] == 'No diferencia significativa').sum()} ({(df_results['Conclusion_ttest'] == 'No diferencia significativa').mean() * 100:.1f}%)",
        f"{(df_results['Significant_KS'] == 'Pass').sum()} ({(df_results['Significant_KS'] == 'Pass').mean() * 100:.1f}%)",
        f"{(df_results['Significant_KS'] == 'Fail').sum()} ({(df_results['Significant_KS'] == 'Fail').mean() * 100:.1f}%)",
        round(df_results['p_val_ttest_uni'].median(skipna=True), 4)
    ]
})

# Mostrar resumen
summary_df

,Metric,Casos (Participación)
0,Total Comparables Iniciales,204
1,"Casos excluidos por 'Rule' ≠ (NO, FK_PRICE)",95
2,Casos excluidos por ausencia de ambas reglas,10
3,Casos comparados finales,99
4,t-test - FK_PRICE < NO,76 (76.8%)
5,t-test - No diferencia significativa,12 (12.1%)
6,KS-test - FK_PRICE ≠ NO,65 (65.7%)
7,KS-test - No diferencia significativa,23 (23.2%)
8,Mediana p-value (t-test),0.0


- Las pruebas estadísticas confirman los hallazgos anteriores. Tenemos en promedio diferencia de medias entre los grupos muestrales de las diferentes combinaciones y los precios promedio compardos en las combinaciones de clases suelen ser inferiores en los productos clasificados como falsificado que los que no.
- *Se puede concluir que la regla de moderación por precio (```FK_PRICE```) excluye productos con precios sospechosamente inferiores a los precios medios de mercado para cada producto en las categorias definidas.*

### Procedemos a entender la regla de moderación por atributos ```FK_ATTRIBUTE```

- Para identificar los patrones de exclusion en FK_ATTRIBUTE se plantea utilizar NLP para hacer conteo de frecuencia de ngrams (1,3) e identificar patrones en los títulos de productos excluidos.

In [17]:
# Asegurar que las stopwords están disponibles
nltk.download('stopwords')

# Cargar stopwords en español
spanish_stopwords = set(stopwords.words('spanish'))

# Agregar stopwords personalizadas
custom_stopwords = {'color', 'gb', 'negro'}  # Personalizar según necesidad
custom_stopwords.update([i.lower() for i in df['Marca'].astype(str)])  # Excluir marcas

# Combinar stopwords en una lista final
all_stopwords = list(spanish_stopwords.union(custom_stopwords))

# Seleccionar dataset
Fattrs = df[df['Rule'] == 'FK_ATTRIBUTE'].set_index('seller_id')['Titulo']
print(f"Total de títulos procesados: {len(Fattrs)}")

# Definir n-gram range (unigramas, bigramas y trigramas)
ngram_range = (1, 3)

# Inicializar CountVectorizer con stopwords personalizadas
vectorizer = CountVectorizer(ngram_range=ngram_range, lowercase=True, stop_words=all_stopwords)

# Transformar los textos a matriz de frecuencia
X = vectorizer.fit_transform(Fattrs)

# Crear un DataFrame con los n-gramas y sus frecuencias
ngram_counts = pd.DataFrame(
    {'ngram': vectorizer.get_feature_names_out(), 'CountVectorizer': X.sum(axis=0).A1}
).sort_values(by='CountVectorizer', ascending=False)

# Separar unigramas de bigramas/trigramas
unigram_counts = ngram_counts[ngram_counts['ngram'].apply(lambda x: len(x.split()) == 1)]
bigram_trigram_counts = ngram_counts[ngram_counts['ngram'].apply(lambda x: len(x.split()) > 1)]

# Filtrar unigramas con frecuencia menor a 200
unigram_counts = unigram_counts[unigram_counts['CountVectorizer'] >= 200]

# Unir unigramas y bigramas/trigramas
filtered_ngrams = pd.concat([unigram_counts, bigram_trigram_counts])

# Mostrar los 10 términos más frecuentes después del filtro
top_n = 10

# Obtener los 10 términos más frecuentes
top_ngrams = filtered_ngrams.head(top_n)

# Crear gráfico de barras horizontales en Plotly con escala de amarillo a azul
fig = px.bar(
    top_ngrams,
    x="CountVectorizer",
    y="ngram",
    orientation="h",
    color="CountVectorizer",
    color_continuous_scale="gnbu",  # Escala de amarillo a azul
    labels={"CountVectorizer": "Frecuencia de aparición", "ngram": "Término detectado"},
    title="<b>Top 10 n-grams más frecuentes en productos excluidos</b>",
)

# Ajustes de diseño para presentación
fig.update_traces(
    marker=dict(
        line=dict(color="#003366", width=1.5)  # Borde azul oscuro con grosor de 1.5 px
    )
)

fig.update_layout(
    font=dict(family="Trebuchet MS", size=18),
    title=dict(font=dict(size=28), x=0.5),
    xaxis=dict(title="Frecuencia", title_font=dict(size=24)),
    yaxis=dict(title="n-gram", title_font=dict(size=24)),
    margin=dict(l=100, r=50, t=80, b=80),
    legend=dict(orientation="h", yanchor="bottom", y=-0.3, xanchor="center", x=0.5),
    template="plotly_white",
    coloraxis_showscale=False,
)



# Mostrar gráfico interactivo
fig.show()

Total de títulos procesados: 1267


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/andresg/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
/Users/andresg/Documents/Case MELI/venv/lib/python3.12/site-packages/sklearn/feature_extraction/text.py:402: UserWarning:

Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ban', 'boss', 'carolina', 'herrera', 'hugo', 'marca', 'posay', 'ray', 'roche', 'segunda', 'simil'] not in stop_words.



>Como parte del proceso se eliminaron stopwords en español y términos irrelevantes, posteriormente se analizaron n-gramas (unigramas, bigramas y trigramas) con un vounter de frecuencia (CountVectorizer), y finalmente se filtraron términos de baja frecuencia para enfocarse en los más representativos.

>Se identificaron los siguientes términos prohibidos que pueden indicar productos no permitidos:
"imitación", "oem", "primera línea", "calidad premium".

In [18]:
# Terminos baneados
banned = ["imitación", "oem", "primera línea", "calidad premium"]

# Seleccionar dataset a tratar
Fattrs = df[df['Rule']=='FK_ATTRIBUTE']
# Convertir 'Titulo' a minúsculas
Fattrs.loc[:,'Titulo'] = [titulo.lower() for titulo in Fattrs['Titulo'].tolist()]

# Crear una expresión regular para terminos baneadas
pattern = re.compile(r'\b(?:' + '|'.join(banned) + r')\b', re.IGNORECASE)

# Aplicar el filtro
filtered = Fattrs[Fattrs['Titulo'].str.contains(pattern, na=False)]
no_fitered = Fattrs[~Fattrs['element_id'].isin(filtered['element_id'])]

fk = len(Fattrs[Fattrs['Fake']==1])
rb = len(Fattrs[Fattrs['Rollback']==1])
no_fk_rb = len(Fattrs[(Fattrs['Fake']==0)&(Fattrs['Rollback']==0)])
print(f'Total productos filtrados por atributo = {len(Fattrs)}\t Total de fakes= {fk}\t Total de rollbacks= {rb}\t Productos no fake ni rollback = {no_fk_rb}')

fk = len(filtered[filtered['Fake']==1])
rb = len(filtered[filtered['Rollback']==1])
no_fk_rb = len(filtered[(filtered['Fake']==0)&(filtered['Rollback']==0)])
print(f'Productos filtrados por terminos baneados= {len(filtered)}\t Total de fakes= {fk}\t Total de rollbacks= {rb}\t Productos no fake ni rollback = {no_fk_rb}')

fk = len(no_fitered[no_fitered['Fake']==1])
rb = len(no_fitered[no_fitered['Rollback']==1])
no_fk_rb = len(no_fitered[(no_fitered['Fake']==0)&(no_fitered['Rollback']==0)])
print(f'Productos restantes filtrados por atributo = {len(no_fitered)}\t Total de fakes = {fk}\t Total de rollbacks= {rb}\t Productos no fake ni rollback = {no_fk_rb}')

Total productos filtrados por atributo = 1267	 Total de fakes= 1187	 Total de rollbacks= 71	 Productos no fake ni rollback = 9
Productos filtrados por terminos baneados= 773	 Total de fakes= 773	 Total de rollbacks= 0	 Productos no fake ni rollback = 0
Productos restantes filtrados por atributo = 494	 Total de fakes = 414	 Total de rollbacks= 71	 Productos no fake ni rollback = 9
